# 🎬 Federated API Governance — MCP + A2A Discovery Demo

## End-to-end demo: MCP servers, A2A agents, and federated API Center

This notebook deploys a **complete federated API governance architecture** from scratch — 4 resource groups demonstrating how Azure API Management + Azure API Center enable unified discovery across **MCP servers**, **A2A agents**, and **REST APIs**.

### Architecture (all deployed by this notebook)
- **RG 2** — REST→MCP conversion: Weather, Catalog, Order + Calculator APIs with APIM MCP proxies
- **RG 3** — FastMCP containers: Weather, Catalog, Order, Calculator as Container Apps
- **RG 4** — A2A agents: Title, Outline, Summary agents with agent cards + REST APIs
- **RG 1** — **Central API Center** aggregating all APIs from RG 2, 3, 4

### Demo story
| Part | Topic | Description |
|------|-------|-------------|
| 1 | **REST→MCP** | Deploy RG 2 — 4 REST APIs converted to MCP servers via APIM policies |
| 2-3 | **FastMCP Containers** | Deploy RG 3 — 4 containerized MCP servers, build & test |
| 4-5 | **Federation** | Deploy central APIC (RG 1), aggregate from RG 2 + RG 3 |
| 6-7 | **MCP Discovery** | Unified catalog, dynamic invoke, cross-env agent workflow |
| 8-9 | **A2A Agents** | Deploy RG 4, explore A2A agents, aggregate to central |
| 10 | **Unified Governance** | Full catalog: REST + MCP + A2A, metrics summary |

### Prerequisites
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and signed in
- [Python 3.12+](https://www.python.org/) with the project requirements installed
- An Azure subscription with **Contributor** access

---
## Part 1 — Deploy REST→MCP APIs (RG 2)
---

> Deploy 4 REST APIs with APIM MCP proxy conversion — Weather, Catalog, Order, Calculator.  
> Each API is: REST backend → APIM policy-based MCP conversion → API Center registration.

### 0️⃣ Initialize Variables

In [ ]:
import os, sys, json, requests
sys.path.insert(1, '../../shared')
import utils

# Ensure correct subscription context
target_sub = "31613fe0-1e9b-4a97-b771-dc48fbaa0fbb"
utils.run(f'az account set --subscription {target_sub}', "", "")

# ═══════════════════════════════════════════════════════════
#  🔢 DEPLOYMENT INDEX — change this to spin up a new demo
#     while the previous one is still deleting
# ═══════════════════════════════════════════════════════════
IDX = 2   # increment for each new demo run (1, 2, 3, …)

# ── RG 2: REST→MCP conversion (Weather, Catalog, Order, Calculator) ──
rg2_name = f"rg-lab-mcp-demo-{IDX}"
rg2_location = "uksouth"
rg2_apim_name = f"apim-mcp-demo-{IDX}"
rg2_apic_prefix = f"apic-demo-{IDX}"
rg2_deployment_name = "mcp-demo-initial"
rg2_calculator_deployment_name = "mcp-demo-calculator"

# ── RG 3: FastMCP containers deployment ──
rg3_name = f"rg-lab-mcp-containers-{IDX}"
rg3_location = "uksouth"
rg3_apim_name = f"apim-fastmcp-{IDX}"
rg3_apic_prefix = f"apic-fastmcp-{IDX}"
rg3_deployment_name = "fastmcp-containers"

# ── RG 4: A2A agents deployment (Title, Outline, Summary) ──
rg4_name = f"rg-lab-a2a-demo-{IDX}"
rg4_location = "uksouth"
rg4_apim_name = f"apim-a2a-{IDX}"
rg4_apic_prefix = f"apic-a2a-{IDX}"
rg4_deployment_name = "a2a-initial"
rg4_summary_deployment_name = "a2a-add-summary"

# ── RG 1: Central API Center ──
rg1_name = f"rg-lab-apic-central-{IDX}"
rg1_location = "swedencentral"
rg1_apic_name = f"apic-central-{IDX}"
rg1_deployment_name = "central-apic"

utils.print_ok(f"Variables initialized  (deployment index = {IDX})")
print(f"  RG 1 (Central APIC):   {rg1_name} ({rg1_location})")
print(f"  RG 2 (REST→MCP):       {rg2_name} ({rg2_location})")
print(f"  RG 3 (FastMCP):        {rg3_name} ({rg3_location})")
print(f"  RG 4 (A2A Agents):     {rg4_name} ({rg4_location})")

### 1️⃣ Deploy RG 2 infrastructure — REST APIs + MCP servers via APIM policies

Deploys: Log Analytics, App Insights, APIM (Basicv2), API Center + 3 REST APIs (Weather, Catalog, Order) each converted to an MCP server via APIM policy.

> ⏱️ First deployment takes ~5-8 minutes (APIM provisioning). Subsequent runs are incremental.

In [ ]:
# Create RG 2 and deploy REST→MCP infrastructure
utils.create_resource_group(rg2_name, rg2_location)

apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": "Basicv2" },
        "apimName": { "value": rg2_apim_name },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "apicLocation": { "value": rg2_location },
        "apicServiceNamePrefix": { "value": rg2_apic_prefix }
    }
}

with open('params-demo-initial.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {rg2_deployment_name} --resource-group {rg2_name} "
    f"--template-file demo-initial.bicep --parameters params-demo-initial.json",
    f"✅ RG 2 deployment '{rg2_deployment_name}' succeeded",
    f"❌ RG 2 deployment failed"
)

### 2️⃣ Retrieve RG 2 outputs + deploy Calculator add-on

In [ ]:
# Retrieve RG 2 deployment outputs
output = utils.run(f"az deployment group show --name {rg2_deployment_name} -g {rg2_name}", "", "")

has_outputs = False
if output.success and output.json_data:
    outputs = output.json_data.get('properties', {}).get('outputs')
    if outputs:
        has_outputs = True

if has_outputs:
    rg2_apim   = utils.get_deployment_output(output, 'apimServiceName', 'APIM Service')
    rg2_gateway = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway')
    rg2_apic   = utils.get_deployment_output(output, 'apicServiceName', 'API Center')
    rg2_apic_api_env = utils.get_deployment_output(output, 'apicApiEnvironmentName', 'APIC API Env')
    rg2_apic_mcp_env = utils.get_deployment_output(output, 'apicMcpEnvironmentName', 'APIC MCP Env')
    rg2_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    rg2_api_key = rg2_subscriptions[0].get("key")
else:
    utils.print_info("Deployment outputs unavailable — resolving from live resources")
    rg2_apim = rg2_apim_name
    o = utils.run(f'az apim show --name {rg2_apim_name} -g {rg2_name} --query gatewayUrl -o tsv', "APIM Gateway", "")
    rg2_gateway = o.text.strip() if o.success else ""
    o = utils.run(f'az apic list -g {rg2_name} --query "[0].name" -o tsv', "API Center", "")
    rg2_apic = o.text.strip() if o.success else ""
    rg2_apic_api_env = "api"
    rg2_apic_mcp_env = "mcp"
    sub_id = utils.get_current_subscription()
    o = utils.run(
        f'az rest --method POST --url "https://management.azure.com/subscriptions/{sub_id}'
        f'/resourceGroups/{rg2_name}/providers/Microsoft.ApiManagement/service/{rg2_apim_name}'
        f'/subscriptions/subscription1/listSecrets?api-version=2022-08-01" --query primaryKey -o tsv',
        "API Key", "")
    rg2_api_key = o.text.strip() if o.success else ""

print(f"\n📋 RG 2 Resources:")
print(f"  APIM:     {rg2_apim}")
print(f"  Gateway:  {rg2_gateway}")
print(f"  APIC:     {rg2_apic}")
print(f"  API Key:  ****{rg2_api_key[-4:] if rg2_api_key else '????'}")

# Deploy Calculator add-on into the same RG 2
calc_params = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimServiceName": { "value": rg2_apim },
        "apicServiceName": { "value": rg2_apic },
        "apicApiEnvironmentName": { "value": rg2_apic_api_env },
        "apicMcpEnvironmentName": { "value": rg2_apic_mcp_env }
    }
}

with open('params-demo-calculator.json', 'w') as f:
    f.write(json.dumps(calc_params))

output = utils.run(
    f"az deployment group create --name {rg2_calculator_deployment_name} --resource-group {rg2_name} "
    f"--template-file demo-add-calculator.bicep --parameters params-demo-calculator.json",
    f"✅ Calculator add-on deployed in RG 2",
    f"❌ Calculator deployment failed"
)

### 📋 Verify RG 2 APIs — REST + MCP servers in API Center and APIM

In [ ]:
# List all APIs in RG 2's API Center
output = utils.run(
    f'az apic api list -g {rg2_name} -n {rg2_apic} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed APIs in RG 2 API Center", "Failed to list APIs")
if output.success:
    print(output.text)

# Snapshot for later comparison
output = utils.run(f'az apic api list -g {rg2_name} -n {rg2_apic} -o json', "", "")
if output.success and output.json_data:
    rg2_apis = output.json_data
    rg2_rest = sum(1 for api in rg2_apis if api.get('kind') == 'rest')
    rg2_mcp  = sum(1 for api in rg2_apis if api.get('kind') == 'mcp')
    print(f"\n📊 RG 2 Total: {len(rg2_apis)} APIs — {rg2_rest} REST, {rg2_mcp} MCP")

# Show APIM APIs including MCP type
subscription_id = utils.get_current_subscription()
print()
output = utils.run(
    f'az rest --method GET '
    f'--url "https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg2_name}'
    f'/providers/Microsoft.ApiManagement/service/{rg2_apim}/apis?api-version=2024-06-01-preview" '
    f'--query "value[].{{Name:name, DisplayName:properties.displayName, Type:properties.type, Path:properties.path}}" -o table',
    f"Listed APIs on APIM '{rg2_apim}'", "Failed")
if output.success:
    print(output.text)

---
## Part 2 — Deploy FastMCP Containers (RG 3)
---

> Deploy 4 containerized FastMCP servers into a **separate resource group** — simulating a different team/subscription.
> Each server is: Container App → APIM MCP Proxy → API Center registration.

### 3️⃣ Deploy infrastructure + containers via Bicep

| Layer | Resources |
|-------|-----------|
| **Monitoring** | Log Analytics, Application Insights, MCP Dashboard |
| **API Gateway** | API Management (Basicv2) with MCP diagnostics |
| **API Governance** | API Center with MCP environment |
| **Containers** | ACR + ACA Environment + 4 Container Apps |
| **MCP Proxies** | 4× Streamable MCP APIs in APIM |
| **Discoverability** | 4× API Center registrations with VS Code install links |

> ⏱️ First deployment takes ~5-8 minutes (APIM provisioning). Subsequent runs are incremental.

In [ ]:
# Create RG 3 and deploy infrastructure
utils.create_resource_group(rg3_name, rg3_location)

apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": "Basicv2" },
        "apimName": { "value": rg3_apim_name },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "apicLocation": { "value": rg3_location },
        "apicServiceNamePrefix": { "value": rg3_apic_prefix }
    }
}

with open('params-demo-containers.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {rg3_deployment_name} --resource-group {rg3_name} "
    f"--template-file demo-mcp-containers.bicep --parameters params-demo-containers.json",
    f"✅ RG 3 deployment '{rg3_deployment_name}' succeeded",
    f"❌ RG 3 deployment failed"
)

### 4️⃣ Retrieve deployment outputs

In [ ]:
# Try deployment outputs first; fall back to direct resource queries
output = utils.run(f"az deployment group show --name {rg3_deployment_name} -g {rg3_name}", "", "")

has_outputs = False
if output.success and output.json_data:
    outputs = output.json_data.get('properties', {}).get('outputs')
    if outputs:
        has_outputs = True

if has_outputs:
    rg3_apim_service  = utils.get_deployment_output(output, 'apimServiceName', 'APIM Service')
    rg3_gateway_url   = utils.get_deployment_output(output, 'apimGatewayUrl', 'APIM Gateway')
    rg3_apic          = utils.get_deployment_output(output, 'apicServiceName', 'API Center')
    rg3_acr_name      = utils.get_deployment_output(output, 'acrName', 'ACR Name')
    rg3_acr_server    = utils.get_deployment_output(output, 'acrLoginServer', 'ACR Server')
    rg3_weather_url   = utils.get_deployment_output(output, 'weatherMcpUrl', 'Weather Container')
    rg3_catalog_url   = utils.get_deployment_output(output, 'catalogMcpUrl', 'Catalog Container')
    rg3_order_url     = utils.get_deployment_output(output, 'orderMcpUrl', 'Order Container')
    rg3_calculator_url = utils.get_deployment_output(output, 'calculatorMcpUrl', 'Calculator Container')
    rg3_app_insights  = utils.get_deployment_output(output, 'appInsightsName', 'App Insights')
    rg3_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    rg3_api_key = rg3_subscriptions[0].get("key")
else:
    utils.print_info("Deployment outputs unavailable — resolving from live resources")

    rg3_apim_service = rg3_apim_name
    o = utils.run(f'az apim show --name {rg3_apim_name} -g {rg3_name} --query gatewayUrl -o tsv', "APIM Gateway", "")
    rg3_gateway_url = o.text.strip() if o.success else ""

    o = utils.run(f'az apic list -g {rg3_name} --query "[0].name" -o tsv', "API Center", "")
    rg3_apic = o.text.strip() if o.success else ""

    o = utils.run(f'az acr list -g {rg3_name} --query "[0].{{n:name,s:loginServer}}" -o json', "ACR", "")
    if o.success and o.json_data:
        rg3_acr_name = o.json_data["n"]
        rg3_acr_server = o.json_data["s"]

    # Container URLs — may be empty if apps haven't started yet
    for app_name, var_name in [("weather-mcp", "rg3_weather_url"), ("catalog-mcp", "rg3_catalog_url"),
                                ("order-mcp", "rg3_order_url"), ("calculator-mcp", "rg3_calculator_url")]:
        o = utils.run(f'az containerapp show --name {app_name} -g {rg3_name} --query "properties.configuration.ingress.fqdn" -o tsv', "", "")
        fqdn = o.text.strip() if o.success and o.text.strip() else ""
        globals()[var_name] = f"https://{fqdn}" if fqdn else ""

    o = utils.run(f'az monitor app-insights component show -g {rg3_name} --query "[0].name" -o tsv', "App Insights", "")
    rg3_app_insights = o.text.strip() if o.success else ""

    sub_id = utils.get_current_subscription()
    o = utils.run(
        f'az rest --method POST --url "https://management.azure.com/subscriptions/{sub_id}'
        f'/resourceGroups/{rg3_name}/providers/Microsoft.ApiManagement/service/{rg3_apim_name}'
        f'/subscriptions/subscription1/listSecrets?api-version=2022-08-01" --query primaryKey -o tsv',
        "API Key", "")
    rg3_api_key = o.text.strip() if o.success else ""

print(f"\n📋 RG 3 Resources:")
print(f"  APIM:     {rg3_apim_service}")
print(f"  Gateway:  {rg3_gateway_url}")
print(f"  APIC:     {rg3_apic}")
print(f"  ACR:      {rg3_acr_name} ({rg3_acr_server})")
print(f"  API Key:  ****{rg3_api_key[-4:] if rg3_api_key else '????'}")

### 5️⃣ Build and push container images to ACR

In [ ]:
containers = [
    ("weather-mcp",    "src/weather/container"),
    ("catalog-mcp",    "src/product-catalog/container"),
    ("order-mcp",      "src/place-order/container"),
    ("calculator-mcp", "src/calculator/container"),
]

for image_name, context_dir in containers:
    output = utils.run(
        f"az acr build --registry {rg3_acr_name} --image {image_name}:latest "
        f"--file {context_dir}/Dockerfile {context_dir} --no-logs",
        f"✅ Built {image_name}", f"❌ Failed to build {image_name}")
    if not output.success:
        break

### 6️⃣ Update container apps with built images

In [ ]:
for image_name, _ in containers:
    output = utils.run(
        f"az containerapp update --name {image_name} --resource-group {rg3_name} "
        f"--image {rg3_acr_server}/{image_name}:latest",
        f"✅ Updated {image_name}", f"❌ Failed to update {image_name}")
    if not output.success:
        break

---
## Part 3 — Test FastMCP Containers
---

### 🧪 Test all 4 containers — direct and through APIM

MCP Streamable HTTP protocol: POST JSON-RPC → SSE response stream

In [ ]:
def mcp_initialize(endpoint, headers=None):
    """Initialize an MCP session and return the session ID."""
    request = {
        "method": "initialize",
        "params": {
            "protocolVersion": "2025-03-26",
            "capabilities": {},
            "clientInfo": {"name": "demo-client", "version": "1.0"}
        },
        "jsonrpc": "2.0", "id": 0
    }
    h = {"Content-Type": "application/json", "Accept": "application/json, text/event-stream"}
    if headers:
        h.update(headers)
    try:
        resp = requests.post(endpoint, json=request, headers=h, timeout=30)
        session_id = resp.headers.get("Mcp-Session-Id", "")
        resp.close()
        return session_id
    except Exception:
        return ""

def call_mcp(endpoint, tool_name, arguments, label="", extra_headers=None):
    """Call an MCP tool (with auto-initialize) and return the parsed result."""
    h = {"Content-Type": "application/json", "Accept": "application/json, text/event-stream"}
    if extra_headers:
        h.update(extra_headers)

    # Initialize to get session
    session_id = mcp_initialize(endpoint, extra_headers)
    if session_id:
        h["Mcp-Session-Id"] = session_id

    request = {
        "method": "tools/call",
        "params": {"name": tool_name, "arguments": arguments},
        "jsonrpc": "2.0", "id": 1
    }
    response = None
    try:
        response = requests.post(endpoint, json=request, stream=True, timeout=30, headers=h)
        if response.status_code == 200:
            for line in response.iter_lines(decode_unicode=True):
                if line and line.startswith("data:"):
                    data = json.loads(line[5:].strip())
                    if "result" in data and "content" in data["result"]:
                        text = data["result"]["content"][0].get("text", "")
                        try:
                            result = json.loads(text)
                        except json.JSONDecodeError:
                            result = text
                        utils.print_ok(f"  {label}: ✅")
                        return result
            utils.print_ok(f"  {label}: ✅ (no data in stream)")
            return None
        else:
            utils.print_error(f"  {label}: HTTP {response.status_code}")
            return None
    except Exception as e:
        utils.print_error(f"  {label}: {e}")
        return None
    finally:
        if response:
            response.close()

def list_mcp_tools(endpoint, label="", extra_headers=None):
    """List available tools on an MCP endpoint."""
    h = {"Content-Type": "application/json", "Accept": "application/json, text/event-stream"}
    if extra_headers:
        h.update(extra_headers)

    session_id = mcp_initialize(endpoint, extra_headers)
    if session_id:
        h["Mcp-Session-Id"] = session_id

    request = {"method": "tools/list", "params": {}, "jsonrpc": "2.0", "id": 1}
    response = None
    try:
        response = requests.post(endpoint, json=request, stream=True, timeout=30, headers=h)
        if response.status_code == 200:
            for line in response.iter_lines(decode_unicode=True):
                if line and line.startswith("data:"):
                    data = json.loads(line[5:].strip())
                    if "result" in data and "tools" in data["result"]:
                        tools = [t["name"] for t in data["result"]["tools"]]
                        print(f"  {label}: {', '.join(tools)}")
                        return tools
        return []
    except Exception as e:
        utils.print_error(f"  {label}: {e}")
        return []
    finally:
        if response:
            response.close()

utils.print_ok("MCP helper functions defined (with session support)")

In [ ]:
# Test each container — direct call
print("🧪 Direct container tests")
print("=" * 50)

tests = [
    (f"{rg3_weather_url}/weather/mcp",      "get_weather",       {"city": "London"},             "Weather"),
    (f"{rg3_catalog_url}/catalog/mcp",       "search_products",   {"query": "laptop"},            "Catalog"),
    (f"{rg3_order_url}/order/mcp",           "place_order",       {"product_id": "PROD-001", "quantity": 1}, "Order"),
    (f"{rg3_calculator_url}/calculator/mcp", "calculate",         {"operation": "multiply", "a": 7, "b": 8}, "Calculator"),
]

results = {}
for endpoint, tool, args, label in tests:
    result = call_mcp(endpoint, tool, args, f"{label} (direct)")
    results[f"{label}_direct"] = result is not None
    if result:
        print(f"    → {json.dumps(result, indent=2)[:200]}")

# Test through APIM (backend URL already includes /mcp, so no extra /mcp needed)
print(f"\n🧪 APIM proxy tests ({rg3_gateway_url})")
print("=" * 50)
apim_headers = {"api-key": rg3_api_key}

apim_tests = [
    (f"{rg3_gateway_url}/weather-mcp",    "get_weather",       {"city": "Paris"},              "Weather"),
    (f"{rg3_gateway_url}/catalog-mcp",     "search_products",   {"query": "phone"},             "Catalog"),
    (f"{rg3_gateway_url}/order-mcp",       "place_order",       {"product_id": "PROD-002", "quantity": 3}, "Order"),
    (f"{rg3_gateway_url}/calculator-mcp",  "calculate",         {"operation": "add", "a": 42, "b": 17}, "Calculator"),
]

for endpoint, tool, args, label in apim_tests:
    result = call_mcp(endpoint, tool, args, f"{label} (APIM)", extra_headers=apim_headers)
    results[f"{label}_apim"] = result is not None
    if result:
        print(f"    → {json.dumps(result, indent=2)[:200]}")

passed = sum(1 for v in results.values() if v)
print(f"\n📊 Results: {passed}/{len(results)} tests passed")

### 7️⃣ Verify RG 3's local API Center

Each container MCP server was auto-registered in RG 3's own API Center during Bicep deployment.

In [ ]:
output = utils.run(
    f'az apic api list -g {rg3_name} -n {rg3_apic} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed APIs in RG 3 API Center", "Failed")
if output.success:
    print(output.text)

# Snapshot
output = utils.run(f'az apic api list -g {rg3_name} -n {rg3_apic} -o json', "", "")
if output.success and output.json_data:
    rg3_apis = output.json_data
    rg3_mcp = sum(1 for api in rg3_apis if api.get('kind') == 'mcp')
    print(f"\n📊 RG 3 Total: {len(rg3_apis)} APIs — {rg3_mcp} MCP Servers")

---
## Part 4 — Deploy Central API Center (RG 1)
---

> 💡 **This is the key architecture moment.**  
> The central API Center acts as the **single pane of glass** for all APIs and MCP servers across the organization — regardless of which resource group, subscription, or APIM instance hosts them.

In [ ]:
# Create RG 1 and deploy central API Center
utils.create_resource_group(rg1_name, rg1_location)

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apicServiceName": { "value": rg1_apic_name },
        "location": { "value": rg1_location }
    }
}

with open('params-central-apic.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {rg1_deployment_name} --resource-group {rg1_name} "
    f"--template-file demo-central-apic.bicep --parameters params-central-apic.json",
    f"✅ Central API Center deployed in {rg1_location}",
    f"❌ Central API Center deployment failed"
)

if output.success and output.json_data:
    rg1_apic_actual = utils.get_deployment_output(output, 'name', 'Central APIC Name')
    utils.print_info(f"Central APIC: {rg1_apic_actual}")

---
## Part 5 — Aggregate APIs into Central Catalog
---

> Pull APIs from **both distributed API Centers** into the central catalog.  
> This simulates a platform team aggregating APIs from multiple product teams.

In [ ]:
import subprocess

def get_apic_apis(resource_group, apic_name):
    """Fetch all APIs from an API Center instance."""
    output = utils.run(f'az apic api list -g {resource_group} -n {apic_name} -o json', "", "")
    if output.success and output.json_data:
        return output.json_data
    return []

def get_apic_deployments(resource_group, apic_name, api_name):
    """Fetch deployments for an API to get the runtimeUri."""
    output = utils.run(
        f'az apic api deployment list -g {resource_group} -n {apic_name} --api-id {api_name} -o json', "", "")
    if output.success and output.json_data:
        return output.json_data
    return []

def create_api_via_arm(api_name, kind, title, description):
    """Create an API in central APIC using ARM REST API (supports mcp/a2a kinds)."""
    url = (f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg1_name}"
           f"/providers/Microsoft.ApiCenter/services/{rg1_apic_name}/workspaces/default/apis/{api_name}"
           f"?api-version=2024-06-01-preview")
    body = json.dumps({"properties": {"title": title, "kind": kind, "description": description[:200]}})
    body_file = f"_temp_api_{api_name}.json"
    with open(body_file, 'w') as f:
        f.write(body)
    output = utils.run(f'az rest --method PUT --url "{url}" --body @{body_file}', "", "")
    try:
        os.remove(body_file)
    except:
        pass
    return output.success

def create_deployment_via_subprocess(api_name, deployment_id, title, env_name, runtime_uris):
    """Create an API deployment using subprocess to handle complex JSON escaping."""
    server_json = json.dumps({"runtimeUri": runtime_uris})
    args = [
        "az", "apic", "api", "deployment", "create",
        "-g", rg1_name, "-n", rg1_apic_name,
        "--api-id", api_name,
        "--deployment-id", deployment_id,
        "--title", title,
        "--environment-id", f"/workspaces/default/environments/{env_name}",
        "--definition-id", f"/workspaces/default/apis/{api_name}/versions/1-0-0/definitions/default",
        "--server", server_json
    ]
    result = subprocess.run(args, capture_output=True, text=True, shell=True)
    return result.returncode == 0

def register_api_in_central(api, source_rg, source_apic, source_label):
    """Register a single API from a distributed APIC into the central APIC."""
    api_name = api['name']
    kind = api.get('kind', 'rest')
    title = api.get('title', api_name)
    description = api.get('description', '') or api.get('summary', '') or f'{title} from {source_label}'
    env_map = {'mcp': 'mcp', 'a2a': 'a2a', 'rest': 'api'}
    env_name = env_map.get(kind, 'api')

    # CLI --type only supports rest/graphql/grpc/soap. Use ARM REST API for mcp/a2a.
    cli_type_map = {'rest': 'rest', 'graphql': 'graphql', 'grpc': 'grpc', 'soap': 'soap'}
    if kind in cli_type_map:
        cmd = (f'az apic api create -g {rg1_name} -n {rg1_apic_name} --api-id {api_name} '
               f'--title "{title}" --type {cli_type_map[kind]} '
               f'--description "{description[:200]}" ')
        utils.run(cmd, "", "")
    else:
        create_api_via_arm(api_name, kind, title, description)

    # Add version
    utils.run(f'az apic api version create -g {rg1_name} -n {rg1_apic_name} --api-id {api_name} '
              f'--version-id 1-0-0 --title "1.0.0" --lifecycle-stage production', "", "")

    # Add definition
    utils.run(f'az apic api definition create -g {rg1_name} -n {rg1_apic_name} --api-id {api_name} '
              f'--version-id 1-0-0 --definition-id default --title "Default"', "", "")

    # Get runtimeUri from source deployments
    deployments = get_apic_deployments(source_rg, source_apic, api_name)
    runtime_uris = []
    for dep in deployments:
        uris = dep.get('server', {}).get('runtimeUri', [])
        runtime_uris.extend(uris)

    # Register deployment with subprocess to handle JSON escaping
    if runtime_uris:
        deploy_id = f"{api_name}-from-{source_label}"
        deploy_title = f"{title} ({source_label})"
        create_deployment_via_subprocess(api_name, deploy_id, deploy_title, env_name, runtime_uris)

    return runtime_uris

utils.print_ok("Aggregation functions defined (supports REST, MCP, and A2A via ARM REST API)")

### 8️⃣ Sync APIs from RG 2 (mixed REST→MCP) → Central APIC

In [ ]:
# Aggregate from RG 2 (mixed REST→MCP APIs)
rg2_apis = get_apic_apis(rg2_name, rg2_apic)
print(f"📥 Syncing {len(rg2_apis)} APIs from RG 2 ({rg2_apic}) → Central APIC")
print("=" * 60)

rg2_synced = 0
for api in rg2_apis:
    uris = register_api_in_central(api, rg2_name, rg2_apic, "rg2")
    uri_str = uris[0] if uris else "—"
    print(f"  ✅ {api['name']:25} ({api.get('kind','?'):4}) → {uri_str}")
    rg2_synced += 1

print(f"\n📊 Synced {rg2_synced} APIs from RG 2")

### 9️⃣ Sync APIs from RG 3 (FastMCP containers) → Central APIC

In [ ]:
# Aggregate from RG 3 (FastMCP containers)
rg3_apis = get_apic_apis(rg3_name, rg3_apic)
print(f"📥 Syncing {len(rg3_apis)} APIs from RG 3 ({rg3_apic}) → Central APIC")
print("=" * 60)

rg3_synced = 0
for api in rg3_apis:
    uris = register_api_in_central(api, rg3_name, rg3_apic, "rg3")
    uri_str = uris[0] if uris else "—"
    print(f"  ✅ {api['name']:25} ({api.get('kind','?'):4}) → {uri_str}")
    rg3_synced += 1

print(f"\n📊 Synced {rg3_synced} APIs from RG 3")

---
## Part 6 — Unified Discovery from Central API Center
---

> 💡 **Single pane of glass** — query one API Center, discover APIs from all environments.

In [ ]:
# List everything in the central API Center
output = utils.run(
    f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed ALL APIs in Central API Center", "Failed")
if output.success:
    print(output.text)

output = utils.run(f'az apic api list -g {rg1_name} -n {rg1_apic_name} -o json', "", "")
if output.success and output.json_data:
    central_apis = output.json_data
    central_rest = sum(1 for api in central_apis if api.get('kind') == 'rest')
    central_mcp  = sum(1 for api in central_apis if api.get('kind') == 'mcp')
    central_a2a  = sum(1 for api in central_apis if api.get('kind') == 'a2a')
    _rg4 = rg4_synced if 'rg4_synced' in dir() else 0
    sources = f"RG 2 ({rg2_synced}) + RG 3 ({rg3_synced})"
    if _rg4 > 0:
        sources += f" + RG 4 ({_rg4})"
    print(f"\n📊 Central Catalog: {len(central_apis)} APIs — {central_rest} REST, {central_mcp} MCP, {central_a2a} A2A")
    print(f"   Aggregated from: {sources}")

### 🔍 Filter: MCP servers with deployment endpoints from Central Catalog

In [ ]:
# Show MCP servers from central catalog with their deployment endpoints
output = utils.run(
    f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[?kind==\'mcp\'].{{Name:name, Title:title, Kind:kind}}" -o table',
    "MCP servers in Central Catalog", "Failed")
if output.success:
    print(output.text)

# Show deployments with runtime URIs for each MCP server
print("\n🔗 MCP Server Endpoints (from Central Catalog deployments)")
print("=" * 80)
mcp_endpoints = {}
for api in central_apis:
    if api.get('kind') == 'mcp':
        deployments = get_apic_deployments(rg1_name, rg1_apic_name, api['name'])
        for dep in deployments:
            uris = dep.get('server', {}).get('runtimeUri', [])
            source = dep.get('title', '')
            for uri in uris:
                print(f"  {api['name']:25} → {uri}  ({source})")
                mcp_endpoints[api['name']] = uri

print(f"\n📊 {len(mcp_endpoints)} MCP servers discoverable with invocation endpoints")

### 🔌 Dynamic connection — discover and invoke MCP servers from Central Catalog

An agent or client can:
1. Query the central API Center for `kind == 'mcp'`
2. Get the `runtimeUri` from deployments
3. Connect and invoke tools — **without knowing which APIM or RG hosts them**

In [ ]:
# Dynamically discover and invoke MCP servers from the central catalog
print("🔌 Dynamic MCP Discovery + Invocation from Central Catalog")
print("=" * 65)

# Step 1: Discover MCP servers
output = utils.run(f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[?kind==\'mcp\']" -o json', "", "")
discovered_servers = output.json_data if output.success and output.json_data else []
print(f"\n  Step 1: Discovered {len(discovered_servers)} MCP servers in central catalog")

# Step 2: Get runtime URIs
print(f"  Step 2: Resolving runtime endpoints...")
invocable = {}
for api in discovered_servers:
    deps = get_apic_deployments(rg1_name, rg1_apic_name, api['name'])
    for dep in deps:
        for uri in dep.get('server', {}).get('runtimeUri', []):
            invocable[api['name']] = uri
            break  # first deployment

# Step 3: Invoke tools/list on our APIM-hosted servers (both RG 2 and RG 3 gateways)
rg2_gw = rg2_gateway.rstrip('/') if 'rg2_gateway' in dir() else ''
rg3_gw = rg3_gateway_url.rstrip('/')

our_servers = {}
our_headers = {}
external = {}
for k, v in invocable.items():
    if v.startswith(rg3_gw):
        our_servers[k] = v
        our_headers[k] = {"Ocp-Apim-Subscription-Key": rg3_api_key}
    elif rg2_gw and v.startswith(rg2_gw):
        our_servers[k] = v
        our_headers[k] = {"Ocp-Apim-Subscription-Key": rg2_api_key}
    else:
        external[k] = v

print(f"  Step 3: Invoking tools on {len(our_servers)} APIM-hosted servers "
      f"(skipping {len(external)} external 3rd-party servers)\n")

for name, endpoint in sorted(our_servers.items()):
    tools = list_mcp_tools(endpoint, label=name, extra_headers=our_headers.get(name, {}))
    if not tools:
        print(f"  ⚠️  {name}: no tools returned")

if external:
    print(f"\n  📋 External MCP servers (not invoked — require vendor auth):")
    for name, uri in sorted(external.items()):
        print(f"     {name:25} → {uri}")

print(f"\n📊 Discovered {len(invocable)} MCP servers, invoked {len(our_servers)} via APIM gateways")

---
## Part 7 — Agent Workflow: Cross-Environment Tool Chaining
---

> An agent discovers MCP servers from the **central catalog** and chains tools across **multiple APIM instances** — Weather from RG 2, Calculator from RG 3, etc.

### Simulated agent scenario:
1. **Discover** available MCP servers from central API Center
2. **Search catalog** (RG 3's APIM) → find a laptop
3. **Place order** (RG 3's APIM) → order it
4. **Calculate total** (RG 3's APIM) → compute with tax
5. **Check weather** (RG 2's APIM) → for delivery planning

In [ ]:
print("🤖 Agent Workflow — Cross-Environment Tool Chaining")
print("=" * 65)

# Step 1: Agent discovers MCP servers from central catalog
print("\n📋 Step 1: Agent queries central API Center for MCP servers")
mcp_servers = {api['name']: api.get('title','') for api in discovered_servers}
for name, title in sorted(mcp_servers.items()):
    endpoint = invocable.get(name, '—')
    if endpoint.startswith(rg3_gw):
        hosted = "RG3-APIM"
    elif rg2_gw and endpoint.startswith(rg2_gw):
        hosted = "RG2-APIM"
    else:
        hosted = "External"
    print(f"  • {title:30} [{hosted}] → {endpoint[:70]}...")

def get_headers_for(endpoint):
    """Return the correct APIM subscription header for an endpoint."""
    if endpoint.startswith(rg3_gw):
        return {"Ocp-Apim-Subscription-Key": rg3_api_key}
    elif rg2_gw and endpoint.startswith(rg2_gw):
        return {"Ocp-Apim-Subscription-Key": rg2_api_key}
    return {}

# Step 2: Search catalog
print("\n🔍 Step 2: Agent calls catalog MCP → search for 'laptop'")
catalog_ep = invocable.get('catalog-mcp')
if catalog_ep:
    result = call_mcp(catalog_ep, "search_products", {"query": "laptop"}, "Catalog search", get_headers_for(catalog_ep))
    if result:
        print(f"    → Found: {json.dumps(result)[:200]}")

# Step 3: Place order
print("\n📦 Step 3: Agent calls order MCP → place order")
order_ep = invocable.get('order-mcp')
if order_ep:
    result = call_mcp(order_ep, "place_order", {"product_id": "PROD-001", "quantity": 2}, "Place order", get_headers_for(order_ep))
    if result:
        print(f"    → Order: {json.dumps(result)[:200]}")

# Step 4: Calculate total with tax
print("\n🧮 Step 4: Agent calls calculator MCP → compute total with tax")
calc_ep = invocable.get('calculator-mcp')
if calc_ep:
    result = call_mcp(calc_ep, "calculate", {"operation": "multiply", "a": 999.99, "b": 1.08}, "Calculate total", get_headers_for(calc_ep))
    if result:
        total = result.get('result', result) if isinstance(result, dict) else result
        print(f"    → Total with 8% tax: ${total}")

# Step 5: Check weather for delivery
print("\n🌤️ Step 5: Agent calls weather MCP → check delivery weather")
weather_ep = invocable.get('weather-mcp')
if weather_ep:
    result = call_mcp(weather_ep, "get_weather", {"city": "Seattle"}, "Weather check", get_headers_for(weather_ep))
    if result:
        print(f"    → Weather: {json.dumps(result)[:200]}")

print("\n" + "=" * 65)
print("✅ Agent workflow complete — 4 tools chained across APIM gateways via central catalog!")

---
## Part 8 — Deploy & Explore A2A Agents (RG 4)
---

> Deploy **A2A agents** (Agent-to-Agent protocol) alongside REST APIs.  
> Each agent exposes `/.well-known/agent.json` agent cards and communicates via the [A2A protocol](https://google.github.io/A2A/).
> 
> **Architecture**: REST API → APIM proxy → A2A agent (registered in APIC with `kind: a2a`)

> ⏱️ First deployment takes ~5-8 minutes (APIM provisioning). Subsequent runs are incremental.

In [ ]:
# Create RG 4 and deploy A2A agents infrastructure
utils.create_resource_group(rg4_name, rg4_location)

apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": "Basicv2" },
        "apimName": { "value": rg4_apim_name },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "apicLocation": { "value": rg4_location },
        "apicServiceNamePrefix": { "value": rg4_apic_prefix }
    }
}

with open('params-a2a-initial.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {rg4_deployment_name} --resource-group {rg4_name} "
    f"--template-file demo-a2a-initial.bicep --parameters params-a2a-initial.json",
    f"✅ RG 4 deployment '{rg4_deployment_name}' succeeded (Title + Outline agents)",
    f"❌ RG 4 deployment failed"
)

# Retrieve RG 4 deployment outputs
output = utils.run(f"az deployment group show --name {rg4_deployment_name} -g {rg4_name}", "", "")

has_outputs = False
if output.success and output.json_data:
    outputs = output.json_data.get('properties', {}).get('outputs')
    if outputs:
        has_outputs = True

if has_outputs:
    rg4_apim   = utils.get_deployment_output(output, 'apimServiceName', 'APIM Service')
    rg4_apim_gateway = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway')
    rg4_apic   = utils.get_deployment_output(output, 'apicServiceName', 'API Center')
    rg4_apic_api_env = utils.get_deployment_output(output, 'apicApiEnvironmentName', 'APIC API Env')
    rg4_apic_a2a_env = utils.get_deployment_output(output, 'apicA2aEnvironmentName', 'APIC A2A Env')
    rg4_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    rg4_api_key = rg4_subscriptions[0].get("key")
else:
    utils.print_info("Deployment outputs unavailable — resolving from live resources")
    rg4_apim = rg4_apim_name
    o = utils.run(f'az apim show --name {rg4_apim_name} -g {rg4_name} --query gatewayUrl -o tsv', "APIM Gateway", "")
    rg4_apim_gateway = o.text.strip() if o.success else ""
    o = utils.run(f'az apic list -g {rg4_name} --query "[0].name" -o tsv', "API Center", "")
    rg4_apic = o.text.strip() if o.success else ""
    rg4_apic_api_env = "api"
    rg4_apic_a2a_env = "a2a"
    sub_id = utils.get_current_subscription()
    o = utils.run(
        f'az rest --method POST --url "https://management.azure.com/subscriptions/{sub_id}'
        f'/resourceGroups/{rg4_name}/providers/Microsoft.ApiManagement/service/{rg4_apim_name}'
        f'/subscriptions/subscription1/listSecrets?api-version=2022-08-01" --query primaryKey -o tsv',
        "API Key", "")
    rg4_api_key = o.text.strip() if o.success else ""

print(f"\n📋 RG 4 Resources:")
print(f"  APIM:     {rg4_apim}")
print(f"  Gateway:  {rg4_apim_gateway}")
print(f"  APIC:     {rg4_apic}")
print(f"  API Key:  ****{rg4_api_key[-4:] if rg4_api_key else '????'}")

# Deploy Summary agent add-on
summary_params = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimServiceName": { "value": rg4_apim },
        "apicServiceName": { "value": rg4_apic },
        "apicApiEnvironmentName": { "value": rg4_apic_api_env },
        "apicA2aEnvironmentName": { "value": rg4_apic_a2a_env }
    }
}

with open('params-a2a-summary.json', 'w') as f:
    f.write(json.dumps(summary_params))

output = utils.run(
    f"az deployment group create --name {rg4_summary_deployment_name} --resource-group {rg4_name} "
    f"--template-file demo-a2a-add-summary.bicep --parameters params-a2a-summary.json",
    f"✅ Summary agent add-on deployed in RG 4",
    f"❌ Summary agent deployment failed"
)

In [ ]:
# List APIs in the A2A API Center — shows all 3 protocol types
print("📋 APIs registered in RG 4 API Center")
print("=" * 70)

output = utils.run(
    f'az apic api list -g {rg4_name} -n {rg4_apic} '
    f'--query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed APIs in A2A API Center", "Failed")
if output.success:
    print(output.text)

# Count by kind
output = utils.run(f'az apic api list -g {rg4_name} -n {rg4_apic} -o json', "", "")
if output.success and output.json_data:
    rg4_apis = output.json_data
    rg4_rest = sum(1 for api in rg4_apis if api.get('kind') == 'rest')
    rg4_mcp  = sum(1 for api in rg4_apis if api.get('kind') == 'mcp')
    rg4_a2a  = sum(1 for api in rg4_apis if api.get('kind') == 'a2a')
    print(f"\n📊 RG 4: {len(rg4_apis)} APIs — {rg4_rest} REST, {rg4_mcp} MCP, {rg4_a2a} A2A agents")

### 🔍 Explore A2A agents — APIM APIs and agent cards

A2A agents are exposed through APIM just like MCP servers. Each agent serves:
- `/.well-known/agent.json` — **Agent Card** (name, description, skills, capabilities)
- `/` — A2A JSON-RPC endpoint for task submission

In [ ]:
# List APIM APIs in RG 4 — shows the A2A agent APIs alongside REST
print("📋 APIM APIs in RG 4")
print("=" * 70)

# Use ARM REST to get API details including type
import subprocess
token = subprocess.run(["az", "account", "get-access-token", "--query", "accessToken", "-o", "tsv"],
                       capture_output=True, text=True, shell=True).stdout.strip()
apim_url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg4_name}/providers/Microsoft.ApiManagement/service/{rg4_apim}/apis?api-version=2024-06-01-preview"
resp = requests.get(apim_url, headers={"Authorization": f"Bearer {token}"})
if resp.status_code == 200:
    apis = resp.json().get('value', [])
    for api in apis:
        name = api['name']
        display = api['properties'].get('displayName', '')
        path = api['properties'].get('path', '')
        api_type = api['properties'].get('type', 'http')
        print(f"  {name:30} path=/{path:20} type={api_type:8} {display}")
    print(f"\n📊 {len(apis)} APIs in APIM ({rg4_apim})")

# Fetch agent cards from the A2A agents via APIM
print("\n🃏 Fetching A2A Agent Cards via APIM")
print("-" * 70)

a2a_agents = [api for api in rg4_apis if api.get('kind') == 'a2a']
for agent in a2a_agents:
    agent_name = agent['name']
    # Get deployment runtimeUri for this agent
    deps = get_apic_deployments(rg4_name, rg4_apic, agent_name)
    for dep in deps:
        for uri in dep.get('server', {}).get('runtimeUri', []):
            card_url = f"{uri}/.well-known/agent.json"
            try:
                card_resp = requests.get(card_url, timeout=10)
                if card_resp.status_code == 200:
                    card = card_resp.json()
                    print(f"\n  🤖 {card.get('name', agent_name)}")
                    print(f"     Description: {card.get('description', 'N/A')}")
                    print(f"     Version: {card.get('version', 'N/A')}")
                    skills = card.get('skills', [])
                    for skill in skills:
                        print(f"     Skill: {skill.get('name', '')} — {skill.get('description', '')}")
                    caps = card.get('capabilities', {})
                    print(f"     Streaming: {caps.get('streaming', False)}")
                else:
                    print(f"\n  ⚠️ {agent_name}: agent card returned {card_resp.status_code}")
            except Exception as e:
                print(f"\n  ⚠️ {agent_name}: {e}")
            break

---
## Part 9 — Aggregate A2A Agents to Central Catalog
---

> Sync APIs from **RG 4** (A2A agents + REST + MCP) into the **Central API Center**.  
> Uses the same `register_api_in_central()` function — it now routes `kind: a2a` to the A2A environment automatically.

In [ ]:
# Aggregate from RG 4 (A2A agents + REST + MCP)
rg4_apis = get_apic_apis(rg4_name, rg4_apic)
print(f"📥 Syncing {len(rg4_apis)} APIs from RG 4 ({rg4_apic}) → Central APIC")
print("=" * 70)

rg4_synced = 0
for api in rg4_apis:
    uris = register_api_in_central(api, rg4_name, rg4_apic, "rg4")
    uri_str = uris[0] if uris else "—"
    kind = api.get('kind', '?')
    icon = {'a2a': '🤖', 'mcp': '🔌', 'rest': '🌐'}.get(kind, '📦')
    print(f"  {icon} {api['name']:25} ({kind:4}) → {uri_str}")
    rg4_synced += 1

rg4_rest = sum(1 for api in rg4_apis if api.get('kind') == 'rest')
rg4_mcp  = sum(1 for api in rg4_apis if api.get('kind') == 'mcp')
rg4_a2a  = sum(1 for api in rg4_apis if api.get('kind') == 'a2a')
print(f"\n📊 Synced {rg4_synced} APIs from RG 4 ({rg4_rest} REST, {rg4_mcp} MCP, {rg4_a2a} A2A)")

---
## Part 10 — Unified Governance: REST + MCP + A2A in One Catalog
---

> 💡 The **Central API Center** now contains APIs from **all 3 protocol types** across 3 distributed resource groups.  
> This is the single pane of glass for discovering and governing all AI-related APIs.

### What's in the Central Catalog
| Protocol | Discovery | Invocation |
|----------|-----------|------------|
| **REST** | `kind: rest` → OpenAPI specs | Standard HTTP `GET`/`POST` |
| **MCP** | `kind: mcp` → `runtimeUri/mcp` | Streamable MCP `tools/list`, `tools/call` |
| **A2A** | `kind: a2a` → `runtimeUri/.well-known/agent.json` | A2A JSON-RPC `tasks/send` |

In [ ]:
# ── 10a. Full catalog view — all protocols, all regions ──────────────────────

# Refresh the central catalog
output = utils.run(f'az apic api list -g {rg1_name} -n {rg1_apic_name} -o json', "", "")
central_apis = output.json_data if output.success and output.json_data else []

# Categorize by kind
central_rest = [a for a in central_apis if a.get("kind") == "rest"]
central_mcp  = [a for a in central_apis if a.get("kind") == "mcp"]
central_a2a  = [a for a in central_apis if a.get("kind") == "a2a"]

print(f"╔══════════════════════════════════════════════════════╗")
print(f"║   Central API Catalog — Unified Discovery Hub       ║")
print(f"╠══════════════════════════════════════════════════════╣")
print(f"║  Total APIs registered:  {len(central_apis):<26} ║")
print(f"║  ─────────────────────────────────────────────────── ║")
print(f"║  🌐 REST APIs:           {len(central_rest):<26} ║")
print(f"║  🔌 MCP Servers:         {len(central_mcp):<26} ║")
print(f"║  🤖 A2A Agents:          {len(central_a2a):<26} ║")
print(f"╚══════════════════════════════════════════════════════╝")

# Print each API with its protocol badge
print(f"\n{'Kind':<6} {'API Name':<40} {'Title'}")
print("─" * 80)
icons = {"rest": "🌐", "mcp": "🔌", "a2a": "🤖"}
for api in sorted(central_apis, key=lambda a: (a.get("kind", ""), a.get("name", ""))):
    kind = api.get("kind", "?")
    icon = icons.get(kind, "❓")
    print(f"{icon} {kind:<4} {api['name']:<40} {api.get('title', '')}")

### 10b. Dynamic A2A Agent Discovery & Invocation

> An **AI agent** (or client) can discover A2A agents from the central catalog at runtime:  
> 1. Query API Center for APIs with `kind == "a2a"`  
> 2. Get the deployment `runtimeUri` for each agent  
> 3. Fetch the **Agent Card** at `{runtimeUri}/.well-known/agent.json`  
> 4. Invoke the agent via A2A protocol (`tasks/send`)  
>
> All traffic flows through **APIM** — giving you rate limiting, auth, monitoring, and governance for free.

In [ ]:
# ── 10b. Dynamic A2A agent discovery from central catalog ────────────────────

# Step 1: Query central APIC for A2A agents
output = utils.run(
    f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[?kind==\'a2a\']" -o json', "", "")
a2a_agents = output.json_data if output.success and output.json_data else []
print(f"Found {len(a2a_agents)} A2A agents in the central catalog\n")

# Step 2: For each agent, get its deployment runtimeUri
discovered_agents = []

for agent in a2a_agents:
    api_name = agent["name"]
    deployments = get_apic_deployments(rg1_name, rg1_apic_name, api_name)

    for dep in deployments:
        runtime_uri = dep.get("server", {}).get("runtimeUri", [None])[0]
        env_id = dep.get("environmentId", "")
        env_name = env_id.split("/")[-1] if env_id else ""

        if runtime_uri and env_name == "a2a":
            discovered_agents.append({
                "name": api_name,
                "title": agent.get("title", api_name),
                "runtime_uri": runtime_uri
            })
            print(f"🤖 {api_name}")
            print(f"   Runtime: {runtime_uri}")

# Step 3: Fetch Agent Cards (via APIM gateway)
print(f"\n{'═' * 70}")
print(f"Agent Cards (fetched from .well-known/agent.json via APIM)")
print(f"{'═' * 70}")

for agent in discovered_agents:
    card_url = f"{agent['runtime_uri']}/.well-known/agent.json"
    try:
        resp = requests.get(card_url, timeout=10)
        if resp.status_code == 200:
            card = resp.json()
            print(f"\n🤖 {card.get('name', agent['name'])}")
            print(f"   Description: {card.get('description', 'N/A')}")
            print(f"   Version:     {card.get('version', 'N/A')}")
            skills = card.get("skills", [])
            if skills:
                for skill in skills:
                    print(f"   Skill:       {skill.get('name', '?')} — {skill.get('description', '')}")
            caps = card.get("capabilities", {})
            if caps:
                print(f"   Capabilities: streaming={caps.get('streaming', False)}, pushNotifications={caps.get('pushNotifications', False)}")
        else:
            print(f"\n⚠️  {agent['name']}: HTTP {resp.status_code}")
    except Exception as e:
        print(f"\n⚠️  {agent['name']}: {e}")

### 10c. Governance Through APIM — All Protocols, One Gateway

> Every API call — **REST**, **MCP**, or **A2A** — flows through **Azure API Management**.  
> APIM provides **consistent governance** regardless of the protocol:  
>
> | Governance Feature | REST | MCP | A2A |
> |---|---|---|---|
> | **Authentication** | ✅ subscription-key / OAuth2 | ✅ subscription-key / OAuth2 | ✅ subscription-key / OAuth2 |
> | **Rate Limiting** | ✅ per-subscription quotas | ✅ per-subscription quotas | ✅ per-subscription quotas |
> | **Request Logging** | ✅ App Insights | ✅ App Insights | ✅ App Insights |
> | **Token Metrics** | ✅ via emit-token-metric | ✅ via emit-token-metric | ✅ via emit-token-metric |
> | **Circuit Breaking** | ✅ backend pool | ✅ backend pool | ✅ backend pool |
> | **IP Filtering** | ✅ policy | ✅ policy | ✅ policy |
>
> The cell below queries **APIM analytics** to show invocation counts and latency across all protocol types.

In [ ]:
# ── 10c. Governance metrics — APIM analytics across all protocols ────────────

from datetime import datetime, timedelta, timezone

# Query APIM analytics — last 7 days
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(days=7)
time_filter = f"timestamp ge datetime'{start_time.strftime('%Y-%m-%dT%H:%M:%SZ')}' and timestamp le datetime'{end_time.strftime('%Y-%m-%dT%H:%M:%SZ')}'"

access_token = subprocess.run(
    ["az", "account", "get-access-token", "--query", "accessToken", "-o", "tsv"],
    capture_output=True, text=True, shell=True).stdout.strip()
headers = {"Authorization": f"Bearer {access_token}"}

# Query each APIM instance
apim_instances = [
    (rg2_name, rg2_apim, "RG 2 — REST→MCP"),
    (rg3_name, rg3_apim_service, "RG 3 — FastMCP"),
    (rg4_name, rg4_apim, "RG 4 — A2A Agents"),
]

for rg, apim, label in apim_instances:
    report_url = (
        f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg}"
        f"/providers/Microsoft.ApiManagement/service/{apim}"
        f"/reports/byApi?$filter={time_filter}&api-version=2023-09-01-preview"
    )
    resp = requests.get(report_url, headers=headers, timeout=30)
    if resp.status_code == 200:
        reports = resp.json().get("value", [])
        print(f"╔══════════════════════════════════════════════════════════════════════╗")
        print(f"║   {label} — {apim:<47} ║")
        print(f"╠══════════════════════════════════════════════════════════════════════╣")
        if reports:
            print(f"║  {'API Name':<30} {'Calls':>8} {'Success':>8} {'Errors':>8} {'Avg ms':>8} ║")
            print(f"║  {'─'*30} {'─'*8} {'─'*8} {'─'*8} {'─'*8} ║")
            total_calls = 0
            for r in sorted(reports, key=lambda x: x.get("callCountTotal", 0), reverse=True):
                name = r.get("name", "?")[:30]
                calls = r.get("callCountTotal", 0)
                success = r.get("callCountSuccess", 0)
                errors = r.get("callCountFailed", 0) + r.get("callCountOther", 0)
                avg_time = r.get("apiTimeAvg", 0)
                total_calls += calls
                print(f"║  {name:<30} {calls:>8} {success:>8} {errors:>8} {avg_time:>7.0f} ║")
            print(f"║  {'─'*30} {'─'*8} {'─'*8} {'─'*8} {'─'*8} ║")
            print(f"║  {'TOTAL':<30} {total_calls:>8}                          ║")
        else:
            print(f"║  No traffic recorded in the last 7 days                          ║")
        print(f"╚══════════════════════════════════════════════════════════════════════╝\n")
    else:
        print(f"⚠️  {label}: Could not fetch reports (HTTP {resp.status_code})\n")

print(f"✅ APIM provides consistent governance across REST, MCP, and A2A protocols")
print(f"   — subscription keys, rate limits, logging, and analytics — all from one gateway.")

---
## ✅ Demo Summary
---

In [ ]:
print("=" * 70)
print("  🎬  FEDERATED API GOVERNANCE DEMO — SUMMARY")
print("=" * 70)
print()
print("  Architecture (all deployed by this notebook):")
print(f"    RG 2 — REST→MCP:      {rg2_apim} ({rg2_name})")
print(f"    RG 3 — FastMCP:        {rg3_apic} ({rg3_name})")
print(f"    RG 4 — A2A Agents:     {rg4_apic} ({rg4_name})")
print(f"    RG 1 — Central APIC:   {rg1_apic_name} ({rg1_location})")
print()
print("  API Inventory:")
_rg2_apis = rg2_apis if 'rg2_apis' in dir() else []
_rg2_rest = rg2_rest if 'rg2_rest' in dir() else '?'
_rg2_mcp  = rg2_mcp  if 'rg2_mcp'  in dir() else '?'
print(f"    RG 2: {len(_rg2_apis)} APIs ({_rg2_rest} REST, {_rg2_mcp} MCP)")
_rg3_mcp = rg3_mcp if 'rg3_mcp' in dir() else '?'
print(f"    RG 3: {len(rg3_apis) if 'rg3_apis' in dir() else '?'} APIs ({_rg3_mcp} MCP)")
_rg4_rest = rg4_rest if 'rg4_rest' in dir() else '?'
_rg4_mcp  = rg4_mcp  if 'rg4_mcp'  in dir() else '?'
_rg4_a2a  = rg4_a2a  if 'rg4_a2a'  in dir() else '?'
_rg4_apis = rg4_apis if 'rg4_apis' in dir() else []
print(f"    RG 4: {len(_rg4_apis)} APIs ({_rg4_rest} REST, {_rg4_mcp} MCP, {_rg4_a2a} A2A)")
_c_rest = len(central_rest) if isinstance(central_rest, list) else central_rest
_c_mcp  = len(central_mcp)  if isinstance(central_mcp, list)  else central_mcp
_c_a2a  = len(central_a2a)  if isinstance(central_a2a, list)  else central_a2a
print(f"    Central: {len(central_apis)} APIs ({_c_rest} REST, {_c_mcp} MCP, {_c_a2a} A2A)")
print()
print("  Key Takeaways:")
print("    • Central API Center = single pane of glass for REST + MCP + A2A discovery")
print("    • APIM = invocation layer with subscription keys, rate limits, and logging")
print("    • A2A agents are discoverable alongside MCP servers and REST APIs")
print("    • Agent cards provide self-describing metadata for A2A agents")
print("    • Agents discover from Central APIC, invoke through distributed APIMs")
print("    • New APIs auto-registered in local APIC, synced to central on demand")
print("    • Works across subscriptions, resource groups, and regions")
print("=" * 70)

---
## 🗑️ Clean up resources

Uncomment and run to delete **all 4 resource groups** created by this demo.  
Or use [clean-up-resources.ipynb](clean-up-resources.ipynb).

In [ ]:
# Uncomment to delete all demo resources (uses current IDX value):

# for rg in [rg2_name, rg3_name, rg4_name, rg1_name]:
#     utils.run(f"az group delete --name {rg} --yes --no-wait",
#               f"✅ {rg} deletion initiated", f"❌ Failed to delete {rg}")
# print(f"💡 To redeploy, increment IDX to {IDX + 1} in the variables cell")

print("⚠️  Uncomment the lines above to delete all 4 resource groups")